<a href="https://www.kaggle.com/code/hossamahmedsalah/artifitial-neural-networks-msp?scriptVersionId=143601725" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="padding: 35px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden;background-image: url(https://github.com/hossamAhmedSalah/Machine_Learning_MSP/blob/main/Assets/247.jpg?raw=true?)">
<b>
<span style='color:skyblue'>MSP Machine Learning workshop 2023 </span>
</b>
<div>
<span style='color:Salmon'>Artificial Neural Networks (ANN)</span>

</div>

</div>

<br>


<a id="r">Table of content</a>
1) [Importing the MNIST (0️⃣ .. 9️⃣) and Visualisation](#1)
2) [Traditional Machine Learning](#2)
    - [RandomForest with the whole dataset](#2.1)
    - [RandomForest with the PCA reduced dim](#2.2)
3) [ANN](#3)
    - [Create Sequential Model](#3.1)
    - [Add Layers](#3.2)
    - [Compile the model](#3.3)
    - [Train the model](#3.4)

# <h1 id="1" style="color: skyblue">Importing the MNIST (0️⃣ .. 9️⃣) and Visualisation</h1>

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import numpy as np

In [ ]:
# train 
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
# test 
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
# train sahpe and test shape
f'train shape {train.shape}', f'test shape {test.shape}'

In [ ]:
# spliting train to x, y
X = train.drop(columns=['label'])
y = train['label']

In [ ]:
def show_images(n, dataset=X,  MAX_IMGS=300):
    num_cols = 10
    if n % num_cols == 0 and n <= MAX_IMGS:
        images = dataset.iloc[:n].values.reshape(-1, 28, 28)
        num_rows = n // num_cols
        fig, ax = plt.subplots(num_rows, num_cols, figsize=(num_cols, num_rows))
        for i in range(num_rows):
            for j in range(num_cols):
                ax[i, j].imshow(images[i * num_cols + j], cmap='gray')
                ax[i, j].axis('off')
        plt.show()
    else:
        print('Invalid number of images')

In [ ]:
show_images(100)

In [ ]:
def show_digits(digit, dataset=X):
    if digit in range(10):
        digit_indices = np.where(y == digit)[0]
        
        for i in range(50):  # Display the first 50 images of the digit
            plt.subplot(5, 10, i + 1)
            imdata = dataset.iloc[digit_indices[i]].values.reshape(28, 28)
            plt.imshow(imdata, cmap='gray')
            plt.xticks([])
            plt.yticks([])

In [ ]:
show_digits(3)

In [ ]:
def do_pca(n_component, dataset):
    X = StandardScaler().fit_transform(dataset)
    pca = PCA(n_components=n_component)
    x_pca = pca.fit_transform(X)
    return pca, x_pca

In [ ]:
import joblib
def fit_forest(X, y, save = (False, 'model_digitREC'), plot =True):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)
    clf = RandomForestClassifier(n_estimators=130, max_depth=None)
    clf.fit(X_train, y_train)
    # predictions
    y_pred = clf.predict(X_test)
    # scoring
    mat = confusion_matrix(y_test, y_pred)
    if plot:
       plt.figure(figsize=(8,8), dpi=170)
       sns.heatmap(mat, annot=True, linewidths=0.5, cmap='Blues',fmt='d')
       plt.show()
    else:
       print(mat)
    acc = accuracy_score(y_test, y_pred)
    print(acc)
    if save[0]:
        joblib.dump(clf, f'{save[1]}.joblib')
    return acc

# <h1 id="2" style="color:skyblue">Traditional Machine Learning</h1>

## <h2 id="21" style="color:skyblue">RandomForest with the whole dataset</h2>

In [ ]:
fit_forest(X, y)

## <h2 id="22" style="color:skyblue">RandomForest with the PCA reduced dim</h2>

In [ ]:
# Reduce the features to 40 only
pca, X_pca = do_pca(40, X)
# let's try fitting to a forest
fit_forest(X_pca, y)

# <h1 style="color:skyblue" id="3">ANN</h1>

![Alt text](image-1.png)

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [ ]:
from keras.datasets import mnist

nb_classes = 10

# the data, shuffled and split between tran and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train original shape", X_train.shape)
print("y_train original shape", y_train.shape)

## <h2 id="31">Create Sequential Model </h2>

In [ ]:
model = Sequential()

## <h2 id="32">Add Layers</h2>

In [ ]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print("Training matrix shape", X_train.shape)
print("Testing matrix shape", X_test.shape)

In [ ]:
from keras.utils import to_categorical

Y_train = to_categorical(y_train, 10)
Y_test =  to_categorical(y_test, 10)

In [ ]:
model.add(Dense(10, activation='sigmoid', input_shape=(784,)))
'''
We explicitly express in the input_shape argument of the first layer what the input data is like:
 a tensor that indicates that we have 784 features of the model.

The tensor is being defined is (None, 784,). 
'''
model.add(Dense(10, activation='softmax'))
'''
The second layer is a softmax layer of 10 neurons, 
which means that it will return a matrix of 10 probability values representing the 10 possible digits.

Each value will be the probability that the image of the current digit belongs to each one of them. 
'''

## <h2 id="33">Compile the model</h2>

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Print the model summary to see the architecture
model.summary()

## <h2 id="34">Train the model</h2>

In [ ]:
model.fit(X_train, Y_train, batch_size=100, epochs=10)

![Alt text](image-2.png)

In [ ]:
# model2
model2 = Sequential()
model2.add(Dense(512, input_shape=(784,)))
model2.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
                              # of the layer above. Here, with a "rectified linear unit",
                              # we clamp all values below 0 to 0.
                           
model2.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
model2.add(Dense(512))
model2.add(Activation('relu'))
model2.add(Dropout(0.2))
model2.add(Dense(10))
model2.add(Activation('softmax')) # This special "softmax" activation among other things,
                                 # ensures the output is a valid probaility distribution, that is
                                 # that its values are all non-negative and sum to 1.

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
history = model2.fit(X_train, Y_train,
          batch_size=128, epochs=10, verbose=1,
          validation_data=(X_test, Y_test))

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy:', score[1])

In [ ]:
model2.save('mnist2.h5')

In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np
from PIL import ImageOps, ImageFilter



model = load_model('mnist2.h5')


def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))

    # Convert image to grayscale
    img_gray = img.convert('L')

    # Invert the grayscale image
    inverted_image = ImageOps.invert(img_gray)

    # Save the inverted image for debugging purposes
    inverted_image.save('inverted.png')

    # Convert the inverted image to a NumPy array
    inverted_image = np.array(inverted_image)

    # Reshape to support our model input and normalize
    inverted_image = inverted_image.reshape(1, 784)
    inverted_image = inverted_image.astype('float32')

    # Predict the class
    res = model.predict(inverted_image)
    print(res)


    print(np.argmax(res[0]), " , ", max(res[0]))

    return np.argmax(res[0]), max(res[0])

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
       
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")
        
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        a,b,c,d = rect
        rect=(a+4,b+4,c-4,d-4)
        im = ImageGrab.grab(rect)

        # Predict the input image
        digit, acc = predict_digit(im)

        # Display the prediction results on the GUI
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
        

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
       
app = App()
mainloop()


In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image, ImageTk
import numpy as np
from PIL import ImageOps

model = load_model('mnist2.h5')

def predict_digit(img):
    # Resize image to 28x28 pixels
    img = img.resize((28, 28))

    # Convert image to grayscale
    img_gray = img.convert('L')

    # Invert the grayscale image
    inverted_image = ImageOps.invert(img_gray)

    # Save the inverted image for debugging purposes
    inverted_image.save('inverted.png')

    # Convert the inverted image to a NumPy array
    inverted_image = np.array(inverted_image)

    # Reshape to support our model input and normalize
    inverted_image = inverted_image.reshape(1, 784)
    inverted_image = inverted_image.astype('float32')

    # Predict the class
    res = model.predict(inverted_image)
    print(res)


    print(np.argmax(res[0]), " , ", max(res[0]))

    return np.argmax(res[0]), max(res[0])
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg="white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text="Recognise", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W)
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        # Bindings
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()
        rect = win32gui.GetWindowRect(HWND)
        a, b, c, d = rect
        rect = (a + 4, b + 4, c - 4, d - 4)
        im = ImageGrab.grab(rect)

        # Predict the input image
        digit, acc = predict_digit(im)

        # Display the prediction results on the GUI
        self.label.configure(text=str(digit) + ', ' + str(int(acc * 100)) + '%')

        # Show the prediction image in a separate window
        im.show()

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')


app = App()
mainloop()


